# Exercise 2: FUN(!) with Singular Vector Decomposition
*Learning from User-generated Data, CP MMS, JKU Linz 2021*
## Details:
This is Exercise 2, an introduction to Matrix Factorization on example of Singular Vector Decomposition.<br>
You don't need to submit anything for this exercise.<br>

Your goal is to work with this notebook at home: read, run, experiment, make sure you understand what is happening here.<br>
This exercise will be graded basing on your results on the next online test.<br>

The test is (probably) going to be composed of multiple choice and insert-a-number questions and consern materal presented here and on corresponding lecture slides. Concentrate on applications and high-level understanding of algorithms, their pros and cons.<br>

The test may contain questions related to the two previous lectures (Memory-based collaborative filtering, Evaluation).

## Matrix Factorization
Remember our old interaction matrix $R$?<br>
Let us try applying factorization to it.

|        | item_1 | item_2 | item_3 | item_4 | item_5 |
| ---    |   ---  |   ---  |   ---  |   ---  |   ---  |
| user_1 | 3 |   | 2 | 3 | 3 |
| user_2 | 4 | 3 | 4 | 3 |   |
| user_3 | 3 | 2 | 1 | 4 | 4 |
| user_4 |   | 5 | 4 | 3 | 1 |
| user_5 | 5 |   | 3 | 4 |   |


The idea is to turn a huge (as above, but much bigger) interaction matrix $R$ of size:
*    $n * m$ ($n$ - number of users, $m$ - number of items)
    
Into two smaller matrices:
*    $U$ of size $n * f$ to represent users;
*    $V$ of size $m * f$ to represent items;<br>
($f$ - number of latent factors, typically $20<<f<<100$)<br><br>
The $f$ dimensions are not neccesserily interpretable (usually they are not).<br>We can imagine each of the dimensions as a recommendation-related characteristic (e.g. **mainstreamness**).<br>This way element $u_{i,1}$ of matrix $U$ tells us how much User $i$ values this characteristic in items. (how much they value mainstream items)<br>Similarly $v_{j,1}$ of matrix $V$ tells how much the characteristic is applicable to the Item $j$ (how mainstream the item is).

### Why factorize?
* The two new matrices combined (should) take less space than the interaction matrix and thus easier fit into memory;
* Selecting a reasonable $f$ means that we operate with shorter vectors during all kinds of calculations, this decreases computational load during recommendation making online inference easier;
* Matrix factorization compresses sparce information contained in the interaction matrix into an elegant representation and can potentially encode **hidden dependencies**;
* Having both items and users represented in the the same $f$-dimensional space opens new possibilities for recommendation;

## Singular Vector Decomposition
A $n * m$ matrix $R$ can be decomposed into a product of 3 matrices:<br>
$R = U\Sigma V^T$

$U$ -- orthogonal ($n * n$) matrix composed of left singular vectors (it corresponds to users);<br>
$\Sigma$ -- ($n * m$) diagonal matrix, containing singular values;<br>
$V$ -- orthogonal ($m * m$) matrix composed of right singular vectors (it corresponds to items);<br>

### <font color='#666666'>Thin Variant</font> of  Singular Vector Decomposition
As before:<br>
$R = U\Sigma V^T$

We can exploit the fact that $R$ (usually) is not square and cannot have *full rank*.<br>
$k = min(n, m)$

As a result $U$, $\Sigma$ and $V$ have different dimensions:<br>
$U$ -- ($n *$ <font color='red'>$k$</font>) of left singular vectors (it corresponds to users);<br>
$\Sigma$ -- (<font color='red'>$k$</font> $*$ <font color='red'>$k$</font>) square diagonal matrix, containing singular values;<br>
$V$ -- ($m *$ <font color='red'>$k$</font>) of right singular vectors (it corresponds to items);<br>

This is a more efficient approach (requires less memory) and it also makes the following demonstration clearer.



In [45]:
from pandas import DataFrame
import numpy as np
from scipy import linalg

### Demonstration
Our good old interaction matrix.<br>
We will consider only the first three users.<br>Having distinctly $3$ users and $5$ items will help us easier understand how it all works.

In [46]:
data = {
    'user_1' : {'item_1' : 3, 'item_2' : np.nan, 'item_3' : 2, 'item_4' : 3, 'item_5' : 3},
    'user_2' : {'item_1' : 4, 'item_2' : 3, 'item_3' : 4, 'item_4' : 3, 'item_5' : np.nan},
    'user_3' : {'item_1' : 3, 'item_2' : 2, 'item_3' : 1, 'item_4' : 4, 'item_5' : 4},
    'user_4' : {'item_1' : np.nan, 'item_2' : 5, 'item_3' : 4, 'item_4' : 3, 'item_5' : 1},
    'Active_User' : {'item_1' : 5, 'item_2' : np.nan, 'item_3' : 3, 'item_4' : 4, 'item_5' : np.nan}
} # dictionary

df = DataFrame(data).T # items become columns and user rows
a = np.array(df)

a_users = ['user_1','user_2','user_3','user_4','Active_User']
a_items = ['RHCP', 'Radiohead', 'Bob_Marley', 'Pink_Floyd', 'ACDC']

# Leaving a 3 by 5 matrix
b = a[:3,:].copy()

# having such arrays helps to keep track of what you recommend to whom 
b_items = a_items
b_users = a_users[:3]

print(b,
      "\n\nUsers:\n", b_users,
     "\n\nItems:\n", b_items)

[[ 3. nan  2.  3.  3.]
 [ 4.  3.  4.  3. nan]
 [ 3.  2.  1.  4.  4.]] 

Users:
 ['user_1', 'user_2', 'user_3'] 

Items:
 ['RHCP', 'Radiohead', 'Bob_Marley', 'Pink_Floyd', 'ACDC']


### Applying SVD
SVD is included in many linear algebra modules. It is present in both Numpy and Scipy:

In [47]:
# linalg.svd returns us the three desired components: U, s, V(transposed)
U, s, Vh = linalg.svd(b) # scipy version

ValueError: array must not contain infs or NaNs

# THIS ⬆
Dont forget, **SVD requires a fully known matrix**, no NaNs allowed!<br>
Ways to cope:
* Augmentation techniques (see slides);

Or better:
* Choose a different factorization algorithm. Minimize squared error on known values (see slides);

Below we replace NaNs with user biases (and without thinking too much) as a stop gap solution.<br>
<font color='red'>In reality we'd rather not learn values that we inserted into the matrix ourselves.</font>

In [48]:
# Creating an array with user-biases
defaults = np.nanmean(a,axis=1) # we calculate the means of each row and ignore the nan-values totally

for i in range(len(b)):
    b[i][np.isnan(b[i])] = defaults[i] # np.isnan(b[i]) as mask: everywhere where row b[i] has nan we replace 
                                        # it with an default value

print('"Augmented" interaction matrix:\n',b)


"Augmented" interaction matrix:
 [[3.   2.75 2.   3.   3.  ]
 [4.   3.   4.   3.   3.5 ]
 [3.   2.   1.   4.   4.  ]]


### Ok, let's try again:

In [49]:
b # rows: users, columns: items

array([[3.  , 2.75, 2.  , 3.  , 3.  ],
       [4.  , 3.  , 4.  , 3.  , 3.5 ],
       [3.  , 2.  , 1.  , 4.  , 4.  ]])

In [50]:
U, s, Vh = np.linalg.svd(b, full_matrices = False)
print("Initial Matrix (R):\n",
      b,
      "\n\nUsers (U):\n",
      U, # each row represents one user 
      "\n\nData Variance per latent feature:\n",
      s,
      "\n\nSame in a form of a diagonal matrix (Sigma):\n",
      np.diag(s),
      "\n\nItems (already transposed!) (Vh):\n",
      Vh) # When Vh wouldn't be transposed then each row would represent one item


Initial Matrix (R):
 [[3.   2.75 2.   3.   3.  ]
 [4.   3.   4.   3.   3.5 ]
 [3.   2.   1.   4.   4.  ]] 

Users (U):
 [[-0.52131022  0.01639708 -0.8532097 ]
 [-0.6515434  -0.65333981  0.38553637]
 [-0.55111419  0.75688719  0.35127613]] 

Data Variance per latent feature:
 [11.8767332   2.33326291  0.55820492] 

Same in a form of a diagonal matrix (Sigma):
 [[11.8767332   0.          0.        ]
 [ 0.          2.33326291  0.        ]
 [ 0.          0.          0.55820492]] 

Items (already transposed!) (Vh):
 [[-0.49032396 -0.37808896 -0.35362487 -0.481868   -0.5092974 ]
 [-0.12579227 -0.17192794 -0.78159983  0.47860897  0.33860336]
 [ 0.06511011 -0.87273558  0.3350064   0.00373437  0.34907025]]


$R = U\Sigma V^T$

Note, that we do **Thin SVD** here, meaning that $U$ is ($n *$ <font color='red'>$k$</font>) and $V^T$ is (<font color='red'>$k$</font> $* m$); Set the parameter **full_matrices** to **True** for vanilla SVD. (see documentation)<br>

Both Scipy and Numpy implementations return the diagonal matrix $\Sigma$ as an array of diagonal values $s$. How very kind of them.<br>

We can immediately see if SVD works the way we expect it to. Don't forget to convert $s$ to a matrix with *np.diag(s)*.<br>Use *@* for matrix multiplication:

In [ ]:
b_r = U @ np.diag(s) @ Vh # reconstructing the original matrix # @ stands for matrix multiplication np.matmul(a,b)

print("Original matrix:\n",b)
print("\n\nReconstructed matrix:\n", b_r)


### Cool things about SVD
SVD projects all the variance contained in the data onto orthogonal basis of $k$ vectors.<br>
Singular values ($\Sigma$ or $s$) allow us to judge how much variance is "situated" along each vector. It is also acts as weighting for the $k$ dimensions;

| $\Sigma$ |  |  |
|--|--|--|
| **11.9** | 0.0 | 0.0 |
| 0.0 | **2.3** | 0.0  |
| 0.0 | 0.0 | **0.6** |

Basing on that, we can choose $f < k$ (remember $f$?⬆) dimensions to represent the whole data. Choosing dimensions corresponding to lagest singular values we make sure to keep data approximately the same while decreasing its size and maybe even filtering some noise out. $U$ and $V^T$ become ($n *$ <font color='red'>$f$</font>) and (<font color='red'>$f$</font> $* m$) respectively.<br><br>
Let's select only $f = 2$ latent features out of $3$ we got, and check how the matrix will change. **Truncated SVD**.

In [ ]:
f = 2 
b_r_f = U[:,:f] @ np.diag(s[:f]) @ Vh[:f,:] # reconstructed with only f features
# U[:,:f]: simply only use part of U. U[row_slicing, column_slicing]
diff = b - b_r_f

print("Initial matrix:\n",
      b,
      "\n\nMatrix reconstructed with ", f, " features:\n",
      b_r_f.round(decimals=2),
      "\n\nProportion of variance captured:",s[:f].sum() / s.sum(),
     "\n\nDifferences ( std = ",np.std(diff.round(decimals=2)),"):\n", diff.round(decimals=2))

### How do we use it now?
Thanks to matrix factorization we have vector representations of both users ($u_i$) and items ($v_j$) (in the same space).

* Project Users and Items to the latent space and select most informative features (corresponding to the higher variance/singular values) to make overall representation more compact;
* By the definition predicted rating for ($User_i$ & $Item_j$) is $u_i \Sigma  v_j$;
* User/Item representations can still be used to estimate similarity in User-/Item- based **Collaborative Filtering**;
* Now that both users and items can be represented in the same space - we can **directly find closest items for a given user**;


* Bearing in mind that vectors of $U$ and $V^T$ are bases of the new f-dimensional space, we can use them to project new users and new items into that space and recommend them (or to them). <font color='red'>Of course this way our model does not take into account additional variance the new items and users may introduce! In order to include the new data to your model you will have to do SVD again on the new interaction matrix!</font>

In [ ]:
# Projecting a new user to the f-dimensional space
user = np.array([3.  , 2., 1.  , 4.  , 4.  ])

print("Initial matrix:\n",
      b,
      "\n\nU:\n",
      U)

print("\n\nUser: ",user, "projected into the f-dim space:\n", np.diag(s**(-1)) @ Vh @ user.T)
user.T @ Vh.T @ np.diag(s**(-1)) # get same result

In [ ]:
# Projecting a new item to the f-dimensional space
# try projecting a known item and see if the resulting vector is the same as produced by SVD
item = [3., 3.5, 4] # we need exactly 3 items bc rated by the 3 given users ???

print("Initial matrix:\n",
      b,
      "\n\nVh:\n",
      Vh)

print("\n\nItem: ",item, "projected into the f-dim space:\n", item @ U @ np.diag(s**(-1)))


In [ ]:
user2 = [5.  , 2., 3.  , 4.  , 4.  ]
user2_p = np.diag(s**(-1)) @ Vh @ user2
print("\n\nUser: ",user2, "projected into the f-dim space:\n",np.diag(s**(-1)) @ Vh @ user2)

item2 = [1., 1, 1]
item2_p = item2 @ U @ np.diag(s**(-1))
print("\n\nItem: ",item2, "projected into the f-dim space:\n",item2 @ U @ np.diag(s**(-1)))

print("\n\nPredicted score for the two items:\n", user2_p @ np.diag(s) @ item2_p.T)

### Notes:
* In general you don't have to follow the $u_i \Sigma  v_j$ mantra for estimating the rating. SVD gives you vector representations and you are free to interpret them as you wish (make sure you are consistent). For example $\Sigma$ makes us treat dimensions differently depending on how much variance they carry which can be not always desirable;
* SVD becomes classical matrix factorization when we get our two matrices, one for users ($U \Sigma^{1/2}$) and items ($\Sigma^{1/2} V^T$);
* Learning latent factors is a more direct approach, it just splits the interaction matrix into two sets of vectors (matrices), taking into account only known values;
* Try different approaches and decide what works best for you in every particular case;

## Questions and Exercises:
* What would be an easy way to calculate RSME for the 2-factor model we created above? How many lines of code would you need?
* What are benefits of using Matrix Factorization?
* Is it easy to update such a model with new items and users? Is it easier than updating a memory-based model?
* What are disadvantages of using SVD for recommendation?
* What are advantages of learning latent factors (ALS, Gradient Descent) compared to SVD? What is the idea of learning latent factors?

* What would be an easy way to calculate RSME for the 2-factor model we created above? How many lines of code would you need? <br>
lines:
1: import stuff, data<br>
2: transform data to df<br>
3: np.linalg to get S sigma V.T<br>
4: Build diagonal matrix of sigma<br>
5: choose first two columns of S, sigma, V because f=2 <br>
6: Compute R with truncated SVD<br>
7: for loop over all users<br>
8: for loop over all items<br>
9: RMSE-sum<br>
10: out of loop; RMSE formula<br>


* What are benefits of using Matrix Factorization?<br>
We can also track other characteristica of users and items without knowing them explicitly.<br>
in f-dimensional space we can compare users with items by distances. <br>
with a f-dimensional space we consider the most important factors without much loss of information --> less memory needed.

* Is it easy to update such a model with new items and users? Is it easier than updating a memory-based model?
Yes, it is easy to update the model by simple formulas. <br>
Easier? Difficult to answer. You need ALL item/user values which are considered in the f-dimesional space. When new user comes with new item then we would have a problem. <br>
Pros: No need of similarity computing which compares new object with all other objects. 

* What are disadvantages of using SVD for recommendation?
Not interpretable<br>
cold-start problem<br>
nans not allowed/hard to handle<br>
need of other users<br>
Computationally expensive<br>
Compute SVD new when over time too much new users/items.

* What are advantages of learning latent factors (ALS, Gradient Descent) compared to SVD? What is the idea of learning latent factors?<br>
nans can be in user rating matrix<br>
only learns on test set: Makes predictions w_u.T*h_i on the fly when needed (less memory space, no need ) 